In [1]:
from astropy.table import Table
import numpy as np
from myfunction import *

In [2]:
filename = '/Users/vaishalp/Dropbox/Final_data/HIPASS+WISE.csv'
HIPASS = Table.read(filename, format = 'csv')

filename = '/Users/vaishalp/Dropbox/Final_data/HIPASS.WISE.10April2017'
Tom = Table.read(filename, format = 'csv')

filename = '/Users/vaishalp/Dropbox/Final_data/HIPASS.WISE.10April2017_spirals'
Tom_spiral = Table.read(filename, format = 'csv')

#filename = '/Users/vaishalp/Dropbox/Final_data/Nick.csv'
#Nick = Table.read(filename, format = 'csv')

filename = '/Users/vaishalp/Dropbox/Final_data/Nick13sample'
Nick = Table.read(filename, format = 'csv')

filename = '/Users/vaishalp/Dropbox/Final_data/Confused_phot'# This is a list of HICAT sources matched to mutliple sources 
confused = Table.read(filename, format = 'csv')

In [3]:
# prints all the modules that have been loaded
#%who
#making a new table:
new_table = Table()

In [4]:
# prints all the column names
# Tom.info

In [5]:
# Defining the first 3 Columns (Name, 2MASX_number & Other_name:
    #this part will need to be adjusted to all of Nick's catolgue 

Name_1 = np.empty( len(HIPASS['HIPASS']), dtype = ('|S25') )
Masx_number_1 = np.empty( len(HIPASS['HIPASS']), dtype = ('|S25') )
Other_name_1 = np.empty( len(HIPASS['HIPASS']), dtype = ('|S25') )
mask =  np.ma.core.MaskedConstant  # this helps to see if the column,row is empty

i = -1
for row in HIPASS:
    i = 1 + i
    if (type(row['HIPASS']) != mask):
        Name_1[i] = row['HIPASS']
    elif (type(row['Matched_name']) != mask):
        Name_1[i] = row['Matched_name']
    elif (type(row['2MASX_number']) != mask):
        Name_1[i] = '2MASX' + row['2MASX_number']

    if (type(row['2MASX_number']) != mask):
        Masx_number_1[i] = row['2MASX_number']
    else:
        Masx_number_1[i] = ""
        
    if (type(row['Matched_name']) != mask):
        Other_name_1[i] = row['Matched_name']
    else:
        Other_name_1[i] = ''

Name_2 = np.empty( len(Nick['Name'][np.where(Nick['HI_source'] != 'HIPASS')]), dtype = ('|S25') )
Masx_number_2 = np.empty( len(Nick['Name'][np.where(Nick['HI_source'] != 'HIPASS')]), dtype = ('|S25') )
Other_name_2 = np.empty( len(Nick['Name'][np.where(Nick['HI_source'] != 'HIPASS')]), dtype = ('|S25') )

i = -1
for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = 1 + i
    if (type(row['Paturel_PGC']) != mask):
        Name_2[i] = row['Paturel_PGC']
    elif (type(row['Huchtmeier_Name']) != mask):
        Name_2[i] = row['Huchtmeier_Name']
    elif (type(row['Springob_OName']) != mask):
        Name_2[i] = row['Springob_OName']
    elif (type(row['Theureau_NAME']) != mask):
        Name_2[i] = row['Theureau_NAME']
    elif (type(row['AGN_Name']) != mask):
        Name_2[i] = row['AGN_Name']
    else:
        Name_2[i]= '2MASX'+ row['Name']

    if (type(row['Name']) != mask):
        Masx_number_2[i] = row['Name']
    else:
        Masx_number_2[i] = ""
  
    if (type(row['Paturel_PGC']) != mask) and (row['Paturel_PGC'] != Name_2[i]):
        Other_name_2[i] = row['Paturel_PGC']
    elif (type(row['Huchtmeier_Name']) != mask) and (row['Huchtmeier_Name'] != Name_2[i]):
        Other_name_2[i] = row['Huchtmeier_Name']
    elif (type(row['Springob_OName']) != mask) and (row['Springob_OName'] != Name_2[i]):
        Other_name_2[i] = row['Springob_OName']
    elif (type(row['Theureau_NAME']) != mask) and (row['Theureau_NAME'] != Name_2[i]):
        Other_name_2[i] = row['Theureau_NAME']
    elif (type(row['AGN_Name']) != mask) and (row['AGN_Name'] != Name_2[i]):
        Other_name_2[i] = row['AGN_Name']
    else:
        Other_name_2[i] = ""
        
Name_Final = np.ma.concatenate((Name_1,Name_2))
Msax_number_Final = np.ma.concatenate((Masx_number_1,Masx_number_2))
Other_name_Final = np.ma.concatenate((Other_name_1,Other_name_2))
print('the total number of galaxies are:',len(Name_Final))

Name = Table.Column(data = Name_Final , name='Name') 
Msax_number = Table.Column(data = Msax_number_Final , name='Msax_number')
Other_name = Table.Column(data = Other_name_Final, name ='Other_name')
new_table.add_columns([Name, Msax_number, Other_name])

print('done')

the total number of galaxies are: 16127
done


In [6]:
# flagging the HICAT sources have been matched to multiple galaxies
HICAT_confused = np.zeros(len(Name_Final))

i = -1

for row in HIPASS:
    i = i + 1
    mask1 = ( confused['HICAT'] == row['HIPASS'])
    if(np.sum(mask1)) >= 1:
        HICAT_confused[i] = 1
    else:
        HICAT_confused[i] = -99
    
for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    HICAT_confused[i] = -99

print(len(HICAT_confused[np.where(HICAT_confused == 1)]))
HICAT_confused = Table.Column(data = HICAT_confused, name = 'HICAT_confused')
new_table.add_columns([HICAT_confused])

print('there are still ',len(HICAT_confused),' galaxies')

print('done') 

147
there are still  16127  galaxies
done


In [7]:
# Defining the next three Columns (RA,Dec,T_type):
RA_Final = np.zeros(len(Name_Final))
Dec_Final = np.zeros(len(Name_Final))
T_type_Final = np.empty(len(Name_Final), dtype = ('|S3'))
i = -1
for row in HIPASS:
    i = 1 + i
    if new_table['HICAT_confused'][i] == -99:
        if (type(row['Matched_RA']) != mask):
            RA_Final[i], Dec_Final[i] = row['Matched_RA'], row['Matched_DEC']
        elif (type(row['HIPASS_RA']) != mask):
            RA_Final[i], Dec_Final[i] = row['HIPASS_RA'], row['HIPASS_Dec']
        if (type(row['NICK_T_Type']) != mask):
            T_type_Final[i] = row['NICK_T_Type']
        elif type(row['RC3_T']) != mask:
            T_type_Final[i] = row['RC3_T']
        else:
            T_type_Final[i] = ""
    else :
        RA_Final[i], Dec_Final[i], T_type_Final[i] = -99,-99, ""

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    RA_Final[i], Dec_Final[i] = row['RA(Deg)'], row['Dec(Deg)']
    T_type_Final[i] = row['T_Type']
    
RA = Table.Column(data = RA_Final, name ='RA')
Dec = Table.Column(data = Dec_Final, name ='Dec')
T_type = Table.Column(data = T_type_Final, name ='T_type')
new_table.add_columns([RA, Dec, T_type])

print('there are still ',len(RA),' galaxies')

print('done')                              

/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Warning: converting a masked element to nan.


there are still  16127  galaxies
done


In [8]:
#Defining the flags (AGN_flag)
    #Note that the galaxies coming from Nicks sample do not have AGN_flags in this current datafile. Their AGN_flags are
        #in the Nick's large file, but I believe that these galaxies are not agns. They also don't have an H

AGN_flag_Final = np.zeros(len(RA_Final))

i = -1
for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        if type(row['AGN_Name']) != mask :
            AGN_flag_Final[i] = 1
        

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    
    if type(row['AGN_Name']) != mask :
        AGN_flag_Final[i] = 1

AGN_flag = Table.Column(data = AGN_flag_Final, name ='AGN_flag')
new_table.add_columns([AGN_flag])

print('there are still ',len(AGN_flag_Final),' galaxies')


# # adding this flag to the confused source table
# AGN = np.zeros( len(confused['HICAT']) )
# i = -1

# for row in confused:
#     i = i + 1
#     mask1 = ( HIPASS['HIPASS'] == row['HICAT'])
#     if(np.sum(mask1)) == 1:
#         if np.array(HIPASS['AGN_Name'][mask1], dtype =('|S25') ) != [b'0']:
#             AGN[i] = 1

# AGN = Table.Column(data = AGN, name ='AGN_flag')
# confused.add_columns([AGN])
        
print('done')  

there are still  16127  galaxies
done


In [9]:
##### defining the HI data columns(Speak, Sint, HI_v, HI_match_flag)

Speak_Final = np.zeros(len(RA_Final))
Sint_Final = np.zeros(len(RA_Final))
Sint_err_Final = np.zeros(len(RA_Final))
HI_v = np.zeros(len(RA_Final))
HI_match_flag = np.zeros(len(RA_Final))

i = -1

for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        Speak_Final[i], Sint_Final[i], HI_v[i], HI_match_flag[i] = row['Speak'], row['Sint'], row['RVmom'], 1

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i +1 
    
    if (type(row['Paturel_F_HI']) != mask):
        HI_match_flag[i], Sint_Final[i], HI_v[i], Speak_Final[i] = 2, row['Paturel_F_HI'], row['Paturel_VHI'], -99
        Sint_err_Final[i] = errMagFlux(row['Paturel_F_HI'],row['Paturel_e_m21'])
  
    elif(type(row['Huchtmeier_value']) != mask) :
        HI_match_flag[i], Sint_Final[i], HI_v[i], Sint_err_Final[i], Speak_Final[i] = 3, row['Huchtmeier_value'], -99, row['Huchtmeier_e_value'], -99
        
    elif (type(row['Springob_Sobs']) != mask):
        HI_match_flag[i], Sint_Final[i], HI_v[i], Sint_err_Final[i], Speak_Final[i] = 4, row['Springob_Sabs'], row['Springob_HRV'],row['Springob_e_Sabs'], -99
        
    elif  (type(row['master_F_HI']) != mask):
        HI_match_flag[i], Sint_Final[i], HI_v[i], Sint_err_Final[i], Speak_Final[i] = 5, row['master_F_HI'], row['master_V_HI'], row['master_e_F_HI'], -99
    
    else:
        HI_match_flag[i], Sint_Final[i], HI_v[i], Sint_err_Final[i], Speak_Final[i] = 0, -99, -99, -99, -99    
        

Speak = Table.Column(data = Speak_Final, name ='Speak')
Sint = Table.Column(data = Sint_Final, name ='Sint')
HI_v = Table.Column(data = HI_v, name ='HI_v')
HI_match_flag = Table.Column(data = HI_match_flag, name = 'HI_match_flag')

new_table.add_columns([Speak, Sint, HI_match_flag, HI_v])

print('there are still ',len(Speak_Final),' galaxies')


# adding HI data to the confused source table

# i = -1
# Speak_Final_confused = np.zeros( len(confused['HICAT']) )
# Sint_Final_confused = np.zeros( len(confused['HICAT']) )
# HI_v_confused = np.zeros( len(confused['HICAT']) )

# for row in confused:
    
#     i = i + 1
#     mask1 = ( HIPASS['HIPASS'] == row['HICAT'])
#     if(np.sum(mask1)) == 1:
#         Speak_Final_confused[i], Sint_Final_confused[i], HI_v_confused[i] =\
#         HIPASS['Speak'][mask1], HIPASS['Sint'][mask1], HIPASS['RVmom'][mask1]

# Speak_confused = Table.Column(data = Speak_Final_confused, name ='Speak')
# Sint_confused = Table.Column(data = Sint_Final_confused, name ='Sint')
# HI_v_confused = Table.Column(data = HI_v_confused, name ='HI_v')
# confused.add_columns([Speak_confused,Sint_confused,HI_v_confused])
print('done') 

/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Warning: converting a masked element to nan.


there are still  16127  galaxies
done


In [10]:
# defining & calculating the distance, redshift and Sint error of each galaxy (D_L, z, Sint_Err)
D_L_Final = np.zeros(len(Name_Final))
d_D_L_Final = np.zeros(len(Name_Final))
z_Final = np.zeros(len(Name_Final))
d_z_Final = np.zeros(len(Name_Final))

i = -1
for row in HIPASS: 
    i = 1 + i
    if new_table['HICAT_confused'][i] == -99:
        z_Final[i],d_z_Final[i] = veltoz(HI_v[i],Speak_Final[i])
        D_L_Final[i], d_D_L_Final[i] = cosmo_cal(z_Final[i],d_z_Final[i])
        Sint_err_Final[i] = sint_err(D_L_Final[i],Sint_Final[i])

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    D_L_Final[i] = row['D_L(Mpc)']
    d_D_L_Final[i] = -99
    if HI_v[i] != -99:
        z_Final[i] = veltoz_noerr(HI_v[i])
        d_z_Final[i] = -99
    elif row['z_uncorrected'] != mask:
        z_Final[i] = row['z_uncorrected']
        d_z_Final[i] = -99 
    else:
        z_Final[i] = -99
        d_z_Final[i] = -99 
        
D_L = Table.Column(data = D_L_Final, name ='D_L(Mpc)')
D_L_Err = Table.Column(data = d_D_L_Final, name ='D_L_Err')
z = Table.Column(data = z_Final, name = 'z')
z_Err = Table.Column(data = d_z_Final, name = 'z_Err')
Sint_err = Table.Column(data = Sint_err_Final, name ='Sint_err')

new_table.add_columns([Sint_err,D_L, D_L_Err, z, z_Err])

print('there are still ',len(D_L_Final),' galaxies')

## Adding Tom's distances to the mix
D_tom = np.zeros(len(new_table['Name']))

i = -1

for row in new_table:
    i = i + 1
    mask1 = (Tom['HICAT'] == row['Name'] )
    if np.sum(mask1) == 1:
        D_tom[i] = Tom['DL'][mask1]
    else:
        mask1 = (Tom_spiral['Name'] == row['Name'] )
        if np.sum(mask1) == 1:
            D_tom[i] = Tom_spiral['DL'][mask1]
        else:
            D_tom[i] = -99.0

D_tom = Table.Column(data = D_tom, name ='D_tom(Mpc)')
new_table.add_columns([D_tom])
# adding HI data to the confused source table

# i = -1
# D_L_Final_confused = np.zeros(len(confused['HICAT']))
# d_D_L_Final_confused = np.zeros(len(confused['HICAT']))
# z_Final_confused = np.zeros(len(confused['HICAT']))
# d_z_Final_confused = np.zeros(len(confused['HICAT']))
# Sint_err_Final_confused = np.zeros(len(confused['HICAT']))

# for row in confused:
#     i = i + 1
#     z_Final_confused[i],d_z_Final_confused[i] = veltoz(HI_v_confused[i],Speak_Final_confused[i])
#     D_L_Final_confused[i], d_D_L_Final_confused[i] = cosmo_cal(z_Final_confused[i],d_z_Final_confused[i])
#     Sint_err_Final_confused[i] = sint_err(D_L_Final_confused[i],Sint_Final_confused[i])

# D_L_confused = Table.Column(data = D_L_Final_confused, name ='D_L(Mpc)')
# D_L_Err_confused = Table.Column(data = d_D_L_Final_confused, name ='D_L_Err')
# z_confused = Table.Column(data = z_Final_confused, name = 'z')
# z_Err_confused = Table.Column(data = d_z_Final_confused, name = 'z_Err')
# Sint_err_confused = Table.Column(data = Sint_err_Final_confused, name ='Sint_err')
# confused.add_columns([Sint_err_confused,D_L_confused, D_L_Err_confused, z_confused, z_Err_confused])

print('done')     

there are still  16127  galaxies
done


In [11]:
###### Defining all the WISE derived parameters that I want 
LogLw1Su = np.zeros(len(Name_Final))
eLogLw1Su = np.zeros(len(Name_Final))
LogMtoL = np.zeros(len(Name_Final))
logMs = np.zeros(len(Name_Final))
Ms_err = np.zeros(len(Name_Final))
LogL_w1 = np.zeros(len(Name_Final))
LogL_w1_err = np.zeros(len(Name_Final))
LogL_w2 = np.zeros(len(Name_Final))
LogL_w2_err = np.zeros(len(Name_Final))
LogL_w3 = np.zeros(len(Name_Final))
LogL_w3_err = np.zeros(len(Name_Final))
LogL_w4 = np.zeros(len(Name_Final))
LogL_w4_err = np.zeros(len(Name_Final))
w1_mJy = np.zeros(len(Name_Final))
w2_mJy = np.zeros(len(Name_Final))
w3_mJy = np.zeros(len(Name_Final))
w4_mJy = np.zeros(len(Name_Final))

i = -1
print('running...')

for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        mask1 = (Tom['HICAT'] == row['HIPASS'])
        
        if(np.sum(mask1)) == 1:
            LogLw1Su[i], eLogLw1Su[i], LogMtoL[i], logMs[i], Ms_err[i], LogL_w1[i], LogL_w1_err[i],\
            LogL_w2[i], LogL_w2_err[i], LogL_w3[i], LogL_w3_err[i], LogL_w4[i], LogL_w4_err[i], \
            w1_mJy[i],w2_mJy[i],w3_mJy[i], w4_mJy[i]\
            = Tom['LogLw1Su'][mask1], Tom['eLogLw1Su'][mask1], Tom['Log(M/L)'][mask1], Tom['Log(smass)'][mask1], Tom['esmass'][mask1],Tom['LogL_w1'][mask1],Tom['eLogL_w1'][mask1],\
            Tom['LogL_w2'][mask1],Tom['eLogL_w2'][mask1],Tom['LogL_w3'][mask1],Tom['eLogL_w3'][mask1],\
            Tom['LogL_w4'][mask1],Tom['eLogL_w4'][mask1],Tom['w1'][mask1], Tom['w2'][mask1], Tom['w3PaH'][mask1],\
            Tom['w4dust'][mask1]
        else:
            LogLw1Su[i], eLogLw1Su[i], LogMtoL[i], logMs[i], Ms_err[i], LogL_w1[i], LogL_w1_err[i],\
            LogL_w2[i], LogL_w2_err[i], LogL_w3[i], LogL_w3_err[i], LogL_w4[i], LogL_w4_err[i], \
            w1_mJy[i], w2_mJy[i], w3_mJy[i], w4_mJy[i]\
            =-99, -99,-99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99
    else:
        LogLw1Su[i], eLogLw1Su[i], LogMtoL[i], logMs[i], Ms_err[i], LogL_w1[i], LogL_w1_err[i],\
        LogL_w2[i], LogL_w2_err[i], LogL_w3[i], LogL_w3_err[i], LogL_w4[i], LogL_w4_err[i], \
        w1_mJy[i], w2_mJy[i], w3_mJy[i], w4_mJy[i]\
        = -99, -99,-99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99
        
for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    mask1 = (Tom_spiral['Name'] == str(Name_Final[i],'utf-8') )
    if np.sum(mask1) == 1:
        LogLw1Su[i], eLogLw1Su[i], LogMtoL[i], logMs[i], Ms_err[i], LogL_w1[i], LogL_w1_err[i],\
        LogL_w2[i], LogL_w2_err[i], LogL_w3[i], LogL_w3_err[i], LogL_w4[i], LogL_w4_err[i], \
        w1_mJy[i],w2_mJy[i],w3_mJy[i], w4_mJy[i]\
        = Tom_spiral['LogLw1Su'][mask1], Tom_spiral['eLogLw1Su'][mask1], Tom_spiral['Log(M/L)'][mask1], \
        Tom_spiral['Log(smass)'][mask1], Tom_spiral['esmass'][mask1], Tom_spiral['LogL_w1'][mask1],\
        Tom_spiral['eLogL_w1'][mask1], Tom_spiral['LogL_w2'][mask1], Tom_spiral['eLogL_w2'][mask1],\
        Tom_spiral['LogL_w3'][mask1], Tom_spiral['eLogL_w3'][mask1], Tom_spiral['LogL_w4'][mask1],\
        Tom_spiral['eLogL_w4'][mask1],Tom_spiral['w1'][mask1], Tom_spiral['w2'][mask1], Tom_spiral['w3PaH'][mask1],\
        Tom_spiral['w4dust'][mask1]
    else:
        LogLw1Su[i], eLogLw1Su[i], LogMtoL[i], logMs[i], Ms_err[i], LogL_w1[i], LogL_w1_err[i],\
        LogL_w2[i], LogL_w2_err[i], LogL_w3[i], LogL_w3_err[i], LogL_w4[i], LogL_w4_err[i], \
        w1_mJy[i], w2_mJy[i], w3_mJy[i], w4_mJy[i]\
        =-99, -99,-99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99,-99, -99

LogLw1Su = Table.Column(data = LogLw1Su, name ='LogLw1Su')
eLogLw1Su = Table.Column(data = eLogLw1Su, name ='eLogLw1Su')
LogMtoL = Table.Column(data = LogMtoL, name ='Log(M/L)')        
LogMs = Table.Column(data = logMs, name ='LogMs')
LogMs_err = Table.Column(data = Ms_err, name ='LogMs_err')
LogL_w1 = Table.Column(data = LogL_w1, name ='LogL_w1')
LogL_w1_err = Table.Column(data = LogL_w1_err, name = 'LogL_w1_err')
LogL_w2 = Table.Column(data = LogL_w2, name ='LogL_w2')
LogL_w2_err = Table.Column(data = LogL_w2_err, name = 'LogL_w2_err')
LogL_w3 = Table.Column(data = LogL_w3, name ='LogL_w3')
LogL_w3_err = Table.Column(data = LogL_w3_err, name = 'LogL_w3_err')
LogL_w4 = Table.Column(data = LogL_w4, name ='LogL_w4')
LogL_w4_err = Table.Column(data = LogL_w4_err, name = 'LogL_w4_err')
w1_mJy = Table.Column(data = w1_mJy, name = 'W1_mJy')
w2_mJy = Table.Column(data = w2_mJy, name = 'W2_mJy')
w3_mJy = Table.Column(data = w3_mJy, name = 'W3_mJy')
w4_mJy = Table.Column(data = w4_mJy, name = 'W4_mJy')
new_table.add_columns([LogMs, LogMs_err, LogL_w1, LogL_w1_err, LogL_w2, LogL_w2_err, LogL_w3, LogL_w3_err,\
                       LogL_w4, LogL_w4_err, LogLw1Su, eLogLw1Su, LogMtoL, w1_mJy,w2_mJy,w3_mJy,w4_mJy])


print('done')

running...


/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:39: UserWarning: Warning: converting a masked element to nan.


done


In [12]:
# Defining all the WISE Photometry Columns (w?, w?err, w?flag & phot_flag):
w1_Final = np.zeros(len(Name_Final))
w1_err_Final = np.zeros(len(Name_Final))
w1_flag_Final = np.zeros(len(Name_Final))
W1snr_Final = np.zeros(len(Name_Final))
phot_flag = np.zeros(len(Name_Final))
w2_Final = np.zeros(len(Name_Final))
w2_err_Final = np.zeros(len(Name_Final)) 
w2_flag_Final = np.zeros(len(Name_Final))
W2snr_Final = np.zeros(len(Name_Final))
w3_Final = np.zeros(len(Name_Final))
w3_err_Final = np.zeros(len(Name_Final))
w3_flag_Final = np.zeros(len(Name_Final))
W3snr_Final = np.zeros(len(Name_Final))
w4_Final = np.zeros(len(Name_Final))
w4_err_Final = np.zeros(len(Name_Final))
w4_flag_Final = np.zeros(len(Name_Final))
W4snr_Final = np.zeros(len(Name_Final))

color12 = np.zeros(len(RA_Final))
color12_err = np.zeros(len(RA_Final))
color23 = np.zeros(len(RA_Final))
color23_err = np.zeros(len(RA_Final))

i = -1
print('running...')

for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        mask1 = (Tom['HICAT'] == row['HIPASS'])
        if (np.sum(mask1)) == 1:
            w1_Final[i],w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i], phot_flag[i] \
            = Tom['w1best'][mask1], Tom['w1berr'][mask1], Tom['w1f'][mask1], Tom['W1snr'][mask1], 1
        
            w2_Final[i],w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
            = Tom['w2best'][mask1],Tom['w2berr'][mask1], Tom['w2f'][mask1], Tom['W2snr'][mask1]
        
            color12[i], color12_err[i] = Tom['W1-W2'][mask1],0
        
            if type(Tom['w3best'][mask1]) != mask :
                w3_Final[i],w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
                = Tom['w3best'][mask1],Tom['w3berr'][mask1], Tom['w3f'][mask1], Tom['W3snr'][mask1]
            
                color23[i], color23_err[i] = Tom['W2-W3'][mask1],0
                
            else:
                w3_Final[i],w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                color23[i], color23_err[i] = -99, -99
            
            if type(Tom['w4best'][mask1]) != mask:
                w4_Final[i],w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
                = Tom['w4best'][mask1],Tom['w4berr'][mask1], Tom['w4f'][mask1], Tom['W4snr'][mask1]
            else:
                w4_Final[i],w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99
                
    #if w1_Final[i] == 0.0:
        else:
       
            if row['ALLWISE_w1gmag'] != mask :
                w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i] \
                = row['ALLWISE_w1gmag'], row['ALLWISE_w1gerr'], row['ALLWISE_w1gflg'], -99
            
                w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
                = row['ALLWISE_w2gmag'], row['ALLWISE_w2gerr'], row['ALLWISE_w2gflg'], -99
            
                phot_flag[i] = 2   
                color12[i] = w1_Final[i] - w2_Final[i]
                color12_err[i] = np.sqrt(w1_err_Final[i]**2 + w2_err_Final[i]**2)
            
                if row['ALLWISE_w3gmag'] != mask:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
                    = row['ALLWISE_w3gmag'], row['ALLWISE_w3gerr'], row['ALLWISE_w3gflg'], -99   
                
                    color23[i] = w2_Final[i] - w3_Final[i]
                    color23_err[i] = np.sqrt(w2_err_Final[i]**2 + w3_err_Final[i]**2)
                else:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                
                    color23[i], color23_err[i] = -99, -99
                if row['ALLWISE_w4gmag'] != mask:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
                    = row['ALLWISE_w4gmag'], row['ALLWISE_w4gerr'], row['ALLWISE_w4gflg'], -99
                else:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99
       
            elif row['ALLSKY_w1gmag'] != mask :
                w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i] \
                = row['ALLSKY_w1gmag'], row['ALLSKY_w1gerr'],  row['ALLSKY_w1gflg'], -99
                w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
                = row['ALLSKY_w2gmag'], row['ALLSKY_w2gerr'],  row['ALLSKY_w2gflg'], -99
            
                phot_flag[i] = 3 
                color12[i] = w1_Final[i] - w2_Final[i]
                color12_err[i] = np.sqrt(w1_err_Final[i]**2 + w2_err_Final[i]**2)
            
                if row['ALLSKY_w3gmag'] != mask:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
                    = row['ALLSKY_w3gmag'], row['ALLSKY_w3gerr'], row['ALLSKY_w3gflg'], -99   
                
                    color23[i] = w2_Final[i] - w3_Final[i]
                    color23_err[i] = np.sqrt(w2_err_Final[i]**2 + w3_err_Final[i]**2)
                else:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                    color23[i], color23_err[i] = -99, -99
                if row['ALLSKY_w4gmag'] != mask:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
                    = row['ALLSKY_w4gmag'], row['ALLSKY_w4gerr'], row['ALLSKY_w4gflg'], -99   
                else:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
                    = -99, -99, -99, -99
       
            elif row['w1mpro'] != mask :
                w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i] \
                = row['w1mpro'], row['w1sigmpro'], -99, -99
                w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
                = row['w2mpro'], row['w2sigmpro'], -99, -99
            
                phot_flag[i] = 4
                color12[i] = w1_Final[i] - w2_Final[i]
                color12_err[i] = np.sqrt(w1_err_Final[i]**2 + w2_err_Final[i]**2)
            
                if row['w3mpro'] != mask:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
                    = row['w3mpro'], row['w3sigmpro'], -99, -99  
                
                    color23[i] = w2_Final[i] - w3_Final[i]
                    color23_err[i] = np.sqrt(w2_err_Final[i]**2 + w3_err_Final[i]**2)
                else:
                    w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                    color23[i], color23_err[i] = -99, -99
                if row['w4mpro'] != mask:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i]  \
                    = row['w4mpro'], row['w4sigmpro'], -99, -99
                else:
                    w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99
        
            else:
                w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i],\
                w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i]\
                = -99, -99, -99, -99, -99, -99, -99, -99
            
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i],\
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = \
                -99, -99, -99, -99, -99, -99, -99, -99
            
                color12[i], color12_err[i], color23[i], color23_err[i] = -99, -99, -99, -99
            
print('done with HIPASS galaxies')


for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    mask1 = (Tom_spiral['Name'] == str(Name_Final[i],'utf-8') )
    if np.sum(mask1) == 1:
        w1_Final[i],w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i], phot_flag[i] \
        = Tom_spiral['w1best'][mask1], Tom_spiral['w1berr'][mask1], Tom_spiral['w1f'][mask1],\
        Tom_spiral['W1snr'][mask1], 1
        
        w2_Final[i],w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
        = Tom_spiral['w2best'][mask1],Tom_spiral['w2berr'][mask1], Tom_spiral['w2f'][mask1], Tom_spiral['W2snr'][mask1]
        
        color12[i], color12_err[i] = Tom_spiral['W1-W2'][mask1],0
        
        if type(Tom_spiral['w3best'][mask1]) != mask :
            w3_Final[i],w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
            = Tom_spiral['w3best'][mask1],Tom_spiral['w3berr'][mask1], Tom_spiral['w3f'][mask1], Tom_spiral['W3snr'][mask1]
            
            color23[i], color23_err[i] = Tom_spiral['W2-W3'][mask1],0
                
        else:
            w3_Final[i],w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
            color23[i], color23_err[i] = -99, -99
            
        if type(Tom_spiral['w4best'][mask1]) != mask:
            w4_Final[i],w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
            = Tom_spiral['w4best'][mask1],Tom_spiral['w4berr'][mask1], Tom_spiral['w4f'][mask1], Tom_spiral['W4snr'][mask1]
        else:
            w4_Final[i],w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99

    else:
        if row['ALLWISE_w1gmag'] != mask :
            w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i] \
            = row['ALLWISE_w1gmag'], row['ALLWISE_w1gerr'], row['ALLWISE_w1gflg'], -99
        
            w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] \
            = row['ALLWISE_w2gmag'], row['ALLWISE_w2gerr'], row['ALLWISE_w2gflg'], -99
        
            phot_flag[i] = 2
            color12[i] = w1_Final[i] - w2_Final[i]
            color12_err[i] = np.sqrt(w1_err_Final[i]**2 + w2_err_Final[i]**2)
        
            if row['ALLWISE_w3gmag'] != mask:
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] \
                = row['ALLWISE_w3gmag'], row['ALLWISE_w3gerr'], row['ALLWISE_w3gflg'], -99 
            
                color23[i] = w2_Final[i] - w3_Final[i]
                color23_err[i] = np.sqrt(w2_err_Final[i]**2 + w3_err_Final[i]**2)
            else:
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                color23[i], color23_err[i] = -99, -99
            if row['ALLWISE_w4gmag'] != mask:
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] \
                = row['ALLWISE_w4gmag'], row['ALLWISE_w4gerr'], row['ALLWISE_w4gflg'], -99
            else:
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99
    
        elif row['w1_gmag'] != mask :
            w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i] = row['w1_gmag'], -99, row['w1_flag'], -99
            w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] = row['w2gmag'], -99, row['w2flag'], -99
            phot_flag[i] = 3
            color12[i] = w1_Final[i] - w2_Final[i]
            color12_err[i] = np.sqrt(w1_err_Final[i]**2 + w2_err_Final[i]**2)
        
            if row['w3_gmag'] != mask:
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = row['w3_gmag'],-99, row['w3_flag'], -99  
                color23[i] = w2_Final[i] - w3_Final[i]
                color23_err[i] = np.sqrt(w2_err_Final[i]**2 + w3_err_Final[i]**2)
            else:
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i] = -99, -99, -99, -99
                color23[i], color23_err[i] = -99, -99
            if row['w4gmag'] != mask:
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i]  = row['w4gmag'], -99, row['w4flag'], -99   
            else:
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99
            
        else:
                w1_Final[i], w1_err_Final[i], w1_flag_Final[i], W1snr_Final[i], \
                w2_Final[i], w2_err_Final[i], w2_flag_Final[i], W2snr_Final[i] = -99, -99, -99, -99, -99, -99, -99, -99
                w3_Final[i], w3_err_Final[i], w3_flag_Final[i], W3snr_Final[i], \
                w4_Final[i], w4_err_Final[i], w4_flag_Final[i], W4snr_Final[i] = -99, -99, -99, -99, -99, -99, -99, -99
                color12[i], color12_err[i], color23[i], color23_err[i] = -99, -99, -99, -99


w1 = Table.Column(data = w1_Final, name ='w1')
w1_err = Table.Column(data = w1_err_Final, name ='w1_err')
w1_flag = Table.Column(data =  w1_flag_Final, name = ' w1_flag')
W1snr = Table.Column(data = W1snr_Final, name = 'w1snr')

w2 = Table.Column(data = w2_Final, name ='w2')
w2_err = Table.Column(data = w2_err_Final, name ='w2_err')
w2_flag = Table.Column(data =  w2_flag_Final, name = ' w2_flag')
W2snr = Table.Column(data = W2snr_Final, name = 'w2snr')

w3 = Table.Column(data = w3_Final, name ='w3')
w3_err = Table.Column(data = w3_err_Final, name ='w3_err')
w3_flag = Table.Column(data =  w3_flag_Final, name = ' w3_flag')
W3snr = Table.Column(data = W3snr_Final, name = 'w3snr')

w4 = Table.Column(data = w4_Final, name ='w4')
w4_err = Table.Column(data = w4_err_Final, name ='w4_err')
w4_flag = Table.Column(data =  w4_flag_Final, name = ' w4_flag')
W4snr = Table.Column(data = W4snr_Final, name = 'w4snr')

color12 = Table.Column(data = color12, name = 'W1-W2')
color12err = Table.Column(data = color12_err, name = 'W1-W2_err')
color23 = Table.Column(data = color23, name = 'W2-W3')
color23err = Table.Column(data = color23_err, name = 'W2-W3_err')

print(len(phot_flag[np.where(phot_flag == 1)]))
phot_flag = Table.Column(data = phot_flag, name = 'phot_flag')
new_table.add_columns([w1, w1_err, w1_flag, W1snr, w2, w2_err, w2_flag, W2snr, w3, w3_err, w3_flag, W3snr, \
                       w4, w4_err, w4_flag, W4snr, phot_flag, color12, color12err, color23, color23err])


print('there are still ',len(w1_Final),' galaxies')

print('done') 

running...


/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:42: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:49: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:37: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:109: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:117: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:102: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5

done with HIPASS galaxies


/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:180: UserWarning: Warning: converting a masked element to nan.
/Users/vaishalp/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:151: UserWarning: Warning: converting a masked element to nan.


3310
there are still  16127  galaxies
done


In [13]:
#Defining HIPASS_match flag, 0 = HICAT galaxy not match, 1 = position and vel matched, 2 position matched, 3 = matched but not sure what type of match it was
HIPASS_match_Final = np.zeros(len(RA_Final))
i = -1
for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        if row['match_flag'] == 0 :
            HIPASS_match_Final[i] = 0

        elif row['match_flag'] == 1 :
            HIPASS_match_Final[i] = 1

        elif row['match_flag'] == 2 :
            HIPASS_match_Final[i] = 2

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    HIPASS_match_Final[i] = -99
    
HIPASS_match_flag = Table.Column(data = HIPASS_match_Final, name = 'HIPASS_match_flag')
    
new_table.add_columns([HIPASS_match_flag])

print('there are still ',len(HIPASS_match_flag),' galaxies')

# adding HI data to the confused source table
# i = -1
# HIPASS_match_Final_confused = np.zeros( len(confused['HICAT']) )

# for row in confused:
#     i = i + 1
#     mask1 = ( HIPASS['HIPASS'] == row['HICAT'])
#     if(np.sum(mask1)) == 1:
#         HIPASS_match_Final_confused[i] = HIPASS['match_flag'][mask1]
# HIPASS_match_flag_confused = Table.Column(data = HIPASS_match_Final_confused, name = 'HIPASS_match_flag')
# confused.add_columns([HIPASS_match_flag_confused])


print('done') 

there are still  16127  galaxies
done


In [14]:
#defining a Nick flag (aka is this galaxy part of Nick's 13000 galaxy sample) 1- yes:
from timeit import default_timer
start = default_timer()
Nick_Final = np.zeros(len(RA_Final))
print('running...')
i = -1
# for row in HIPASS:
#     i = i + 1
#     Nick_Final[i] = -99
#     for rowt in Nick:
#         if row['2MASX_number'] == rowt['Name']:
#             Nick_Final[i] = 1
#             print(i)

for row in HIPASS:
    i = i + 1
    if new_table['HICAT_confused'][i] == -99:
        if row['2MASX_number'] != mask:
            mask1 = (Nick['Name'] == row['2MASX_number'])
            if np.sum(mask1) == 1:
                Nick_Final[i] = 1
            else:
                Nick_Final[i] = -99
        else:
            Nick_Final[i] = -99
#     print(Nick_Final[i])
print('done with HIPASS galaxies')

for row in Nick[np.where(Nick['HI_source'] != 'HIPASS')]:
    i = i + 1
    Nick_Final[i] = 1

Nick_flag = Table.Column(data = Nick_Final, name = 'Nick_Flag')
    
new_table.add_columns([Nick_flag])

print('there are still ',len(Nick_Final),' galaxies')
print( default_timer() - start)
print('done') 

running...
done with HIPASS galaxies
there are still  16127  galaxies
1.2282038549965364
done


In [15]:
#Creating the New Table
new_table.write('/Users/vaishalp/Dropbox/Final_data/HIPASS.WISE.29Dec2017_all.csv', format='csv', overwrite=True)


# confused.write('/Users/vaishalp/Dropbox/Final_data/Confused_phot', format = 'csv')
# print('done!:)')